# LangGraph is an LLM scheduling tool associate with LangChain

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import gradio as gr
import random
from dotenv import load_dotenv

load_dotenv(override=True)

Using MemorySaver we can get the Agents to have memory between prompts:

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

Create a small serper tool:

In [ ]:
# from langchain.agents import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.tools import tool

serper = GoogleSerperAPIWrapper()

@tool
def tool_search(search_query: str) -> str:
    """A search tool using Serper API.

    Args:
        search_query: The query string to search for.
    """
    return serper.run(search_query)

print(tool_search.run("Muscle relaxation techniques for stress relief"))

All tools must be placed into a tools list

In [ ]:
tools = [tool_search]

LangGraph needs us to set up a state class:

In [ ]:
class State(BaseModel):
    messages: Annotated[list, add_messages]

Then we can initialise the graphbuilder

In [ ]:
graph_builder = StateGraph(State)

Define the LLM and add nodes:

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot_node(old_state: State) -> State:
    response = llm_with_tools.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_node("tools", ToolNode(tools=tools))

Build the edges of the graph:

In [ ]:
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

Compile the graph:

In [ ]:
graph = graph_builder.compile(checkpointer=memory)

Then run the graph, config is required for memory

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def chat(user_input: str, history):
    initial_state = State(messages=[{"role": "user", "content": user_input}])
    result = graph.invoke(initial_state, config=config)
    print(result)
    return result['messages'][-1].content

gr.ChatInterface(chat).launch()